# A Comparative Study of Statistical-based Change Detection Methods for Multidimensional and Multitemporal SAR Images

Created by: Ammar Mian, SONDRA, CentraleSupélec
Date: 24/06/2019
Contact: ammar.mian@centralesupelec.fr

This notebook aims at giving reproducible code for the World Congress on Condition monitoring 2019 paper on Change detection. System-requirements:
* Python 3
* Scipy, numpy
* tqdm
* matplotlib and plotly

The simulation computes change detection maps for each statistics function in the **statistic_list** list. The definition of those functions can be found in the file **change_detection_functions.py**.


In [ ]:
# Importing all needed libraries and functions

import numpy as np
import time

# Plotting
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode()

# Custom functions
from generic_functions import *
from multivariate_images_tools import *
from data_management import *
from change_detection_functions import *
from monte_carlo_tools import compute_several_statistics

## Simulation settings

In [ ]:
# Enable parallel processing (or not)
enable_multi = True
# These two variables serves to split the original image into sub-images to be treated in parallel
# In general the optimal parameters are obtained for 
# number_of_threads_rows*number_of_threads_columns = number of cores on the machine
number_of_threads_rows = 2
number_of_threads_columns = 2

# Plotting backend is either Matplotlib or Plotly
plotting_beckend = 'Matplotlib'

# Dataset
dataset_choice = 'UAVSAR Scene 1'
path_to_data = '../Data/'

# List of statistics to compute
statistic_list = [covariance_equality_glrt_gaussian_statistic, 
                 covariance_equality_t1_gaussian_statistic,
                  covariance_equality_Wald_gaussian_statistic,
                 complex_Hotelling_Lawley_trace_statistic,
                  scale_and_shape_equality_robust_statistic,
                  Kullback_Leibler_statistic,
                  Similarity_measure_Meng_Liu,
                  Natural_Geodesic_statistic,
                Natural_student_t_distance,
                Wasserstein_Gaussian_statistic,
                 Wasserstein_robust_statistic]
statistic_names = [r'Gaussian GLRT', 
                   r't1 statistic',
                   r'Wald statistic',
                   r'HTL statistic',
                   r'MT statistic',
                  r'Kullback Leibler statistic',
                   r'MLL statistic',
                   r'Natural Gaussian Geodesic statistic',
                    r'Natural_student_t_distance',
                   r'Wasserstein Gaussian statistic',
                  r'Wasserstein robust statistic']
args_list = ['log', 'linear', 'log', None, (0.01, 15, 'log'), None, (0.01,15), None, (3,0.01,15),  None, (0.01,15)]
number_of_statistics = len(statistic_list)


# Sliding windows mask used
windows_mask = np.ones((5,5))
m_r, m_c = windows_mask.shape

# For ROC curve
number_of_points = 100

## Reading and plotting dataset

In [ ]:

dataset = fetch_dataset(dataset_choice, path_to_data)
for t in range(dataset.number_dates):
  plot_Pauli_SAR(dataset.data[:,:,:,t], figsize=dataset.figsize)
  plt.title(r'Image at $t=%d$'%t)

if dataset.ground_truth is not None:
    plt.figure(figsize=dataset.figsize, dpi=80, facecolor='w')
    plt.imshow(dataset.ground_truth, aspect='auto')
    plt.title('Ground Truth')
    plt.axis('off')
plt.show()

## Computing change detection maps

In [ ]:
t_beginning = time.time()
function_to_compute = compute_several_statistics
function_args = [statistic_list, args_list]
results = sliding_windows_treatment_image_time_series_parallel(dataset.data, windows_mask, function_to_compute, 
                function_args, multi=enable_multi, number_of_threads_rows=number_of_threads_rows,
                number_of_threads_columns=number_of_threads_columns, progressbar=True)
print('Done.')
print("Elpased time: %d s" %(time.time()-t_beginning))

In [ ]:
# Showing statistics results raw Matplotlib
if plotting_beckend == 'Matplotlib':
    for i_s, statistic in enumerate(statistic_names):
        image_temp = np.nan*np.ones((dataset.number_rows, dataset.number_columns))
        image_temp[int(m_r/2):-int(m_r/2), int(m_c/2):-int(m_c/2)] = (results[:,:,i_s] - results[:,:,i_s].min())
        plt.figure(figsize=dataset.figsize, dpi=80, facecolor='w')
        plt.imshow(image_temp, aspect='auto')
        plt.title(statistic)
        plt.axis('off')
        plt.colorbar()
        plt.savefig('./Results/'+dataset_choice+'/%s_%dx%d.pdf'%(statistic, m_r, m_c) )
    plt.show()

In [ ]:
# Showing statistics results raw Plotly
if plotting_beckend == 'Plotly':
    for i_s, statistic in enumerate(statistic_names):
        image_temp = np.nan*np.ones((dataset.number_rows, dataset.number_columns))
        image_temp[int(m_r/2):-int(m_r/2), int(m_c/2):-int(m_c/2)] = np.flip(results[:,:,i_s] - results[:,:,i_s].min(), axis=0)
        trace = go.Heatmap(z=image_temp, colorscale='Jet')
        fig = go.Figure(data=[trace])
        iplot(fig)

## Computing ROC curves

In [ ]:
t_beginning = time.time()
ground_truth = dataset.ground_truth[int(m_r/2):-int(m_r/2), int(m_c/2):-int(m_c/2)]
pfa_array = np.zeros((number_of_points, len(function_args[0])))
pd_array = np.zeros((number_of_points, len(function_args[0])))
for i_s, statistic in enumerate(tqdm(statistic_names)):

    # Sorting values of statistic
    λ_vec = np.sort(vec(results[:,:,i_s]), axis=0)
    λ_vec = λ_vec[np.logical_not(np.isinf(λ_vec))]

    # Selectionning number_of_points values from beginning to end
    indices_λ = np.floor(np.logspace(0, np.log10(len(λ_vec)-1), num=number_of_points)) # logspace
    # indices_λ = np.floor(np.logspace(0, len(λ_vec)-1, num=number_of_points)) # logspace
    λ_vec = np.flip(λ_vec, axis=0)
    λ_vec = λ_vec[indices_λ.astype(int)]

    # Thresholding and summing for each value
    for i_λ, λ in enumerate(λ_vec):
        good_detection = (results[:,:,i_s] >= λ) * ground_truth
        false_alarms = (results[:,:,i_s] >= λ) * np.logical_not(ground_truth)
        pd_array[i_λ, i_s] = good_detection.sum() / (ground_truth==1).sum()
        pfa_array[i_λ, i_s] = false_alarms.sum() / (ground_truth==0).sum()
print('Done.')
print("Elpased time: %d s" %(time.time()-t_beginning))

In [ ]:
# Showing statistics results ROC with matplotlib
import matplotlib2tikz
markers = ['o', 's', 'h', '*', 'd', 'p', '+', '^', '>', '<']
plt.figure(figsize=(6, 4), dpi=120, facecolor='w')
for i_s, statistic in enumerate(statistic_names):
    plt.semilogx(pfa_array[:,i_s], pd_array[:,i_s], linestyle='--', label=statistic,
        marker=markers[i_s], markersize=4, linewidth=1)
plt.xlabel(r'$\mathrm{P}_{\mathrm{FA}}$')
plt.ylabel(r'$\mathrm{P}_{\mathrm{D}}$')
plt.legend()
plt.xlim([0.,1])
plt.ylim([0,1])
matplotlib2tikz.save('./Results/'+dataset_choice+'/Robust_statistics_%dx%d.tex'%(m_r, m_c))

In [ ]:
# Showing statistics results ROC with plotly
data = []
symbols = ['circle', 'square', 'cross', 'star']
for i_s, statistic in enumerate(statistic_names):
    trace = go.Scatter(x=pfa_array[:,i_s], y=pd_array[:,i_s], name=statistic, mode = 'lines+markers')
    data.append(trace)

layout = go.Layout(
    width=900,
    height=400,
    xaxis=dict(
        title=r'Probability of false alarm',
        type='log',
        autorange=True
    ),
    yaxis=dict(
        autorange=True, 
        title=r'Probability of detection',
    ),
    template='seaborn',
    hovermode= 'x',
    autosize=False,
    font=dict(size=18)
    )
config = {
    'editable': False,
    'scrollZoom': True,
    'displayModeBar': True,
    'showLink': False
}

fig = go.Figure(data=data, layout=layout)
iplot(fig, config=config)